## Process population and reported cases files and gets data per million inhabitants
- Files produced: 
- Cases per million inhabitants by variable and date
- Population by country

In [1]:
import pandas as pd
import numpy as np

In [ ]:
def process_data(data,population_admin):

    population_admin=population.groupby(['NAME_0','NAME_1']).Population_mean.sum().reset_index().rename(columns={'NAME_0':'name_0','NAME_1':'name_1'})

    data.date=pd.to_datetime(data.date)

    data['year']=data.date.dt.year
    data['month']=data.date.dt.month

    data['month_year'] = data['date'].dt.to_period('M')

    data=data.groupby(['name_0', 'name_1','month_year','tar_wrongdoing', 'pe_approxnumber','pe_violence','tar_outcome']).id.count().reset_index()

    data.name_1=data.name_1.replace("Chimaltenullgo","Chimaltenango")
    data.name_1=data.name_1.replace("Huehuetenullgo","Huehuetenango")
    data.name_1=data.name_1.replace("Quetzaltenullgo","Quetzaltenango")

    data=data.merge(population_admin,on=['name_0','name_1'],how='left')

    ## data per million inhabitants
    data['events_pop']=data['id']/(data['Population_mean']/1e6)

    data=data.dropna()

    data['month_year']=data['month_year'].apply(str)

    data['month_year']=data['month_year']+"-01"
    return data

In [5]:
def main():
    data_path="./src/data/"
    data=pd.read_json(data_path+"json_data_complete_latin3.json")
    population=pd.read_csv(data_path+"LatinAmerica_geo_variables_provinces.csv",encoding='latin-1')
    data=process_data(data,population_admin)
    data.to_json(data_path+"data_agg.json",orient='records')
    
    ## population country level
    pop_name0=data.loc[:,['name_0','Population_mean']].drop_duplicates().groupby(['name_0']).Population_mean.sum().reset_index()
    pop_name0=pop_name0.pivot_table(aggfunc=np.sum,columns='name_0')
    pop_name0.to_json(data_path+"population_admin0.json",orient='records')
    
if __name__ == '__main__':
    main()

(3543, 14)